In [1]:
import tqdm
import numpy as np
import pandas as pd
from datetime import datetime
import sys

sys.path.append('../../')
from utils import file_utils, microarray_utils, clustering_utils

/Users/michaelbertagna/anaconda3/envs/debug_tgne.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def min_max_scale_2d_arr(arr: np.array):

    flat_arr = arr.flatten()

    non_inf_mask = flat_arr != float('inf')

    max_val = max(flat_arr[non_inf_mask])
    min_val = min(flat_arr)

    scaled_arr = (arr - min_val) / (max_val - min_val)

    return scaled_arr

In [3]:
clr_df = pd.read_csv('./clr_network_for_distances.csv')
clr_df.rename(columns={'Unnamed: 0':'TTHERM_ID'}, inplace=True)
print(clr_df.shape)
clr_df.head()

In [4]:
max_zscore = clr_df.max(axis=None, numeric_only=True)
min_zscore = clr_df.min(axis=None, numeric_only=True)
zscore_arr = clr_df.loc[:,clr_df.columns[1:]].to_numpy()

info = np.finfo(np.float64)
smallest_float = info.eps

# sys.float_info.min
# sys.float_info.epsilon

scaled_zscore_arr = min_max_scale_2d_arr(zscore_arr)
zero_zscore_idxs = np.where(scaled_zscore_arr == 0)
clr_dist_arr = np.sqrt(2 * (1 - scaled_zscore_arr)) + smallest_float
np.fill_diagonal(clr_dist_arr, 0)

print(min((clr_dist_arr[np.where(clr_dist_arr > 0)]).flatten()))
print(max((clr_dist_arr[np.where(clr_dist_arr > 0)]).flatten()))
print(np.where(clr_dist_arr > 1))
print(np.where(clr_dist_arr == 0))
print(np.all(np.where(clr_dist_arr <= 0)[0] == np.where(clr_dist_arr <= 0)[1]))

In [5]:
partition_type = 'EXP'
full_filtered_df = pd.read_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv')
full_filtered_df = full_filtered_df.rename(columns={'Unnamed: 0': 'TTHERM_ID'})

In [6]:
full_filtered_df.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_000000042,6.928782,7.264201,6.934214,6.732989,6.970612,7.150978,6.126826,6.868968,6.641119,...,6.450318,8.049750,7.788162,7.052154,6.517742,6.918501,6.048861,7.041619,6.757932,5.817246
1,TTHERM_000000045,9.633489,9.977124,10.027529,9.720665,9.605762,10.225542,10.279608,10.459966,10.693337,...,11.130466,11.207738,11.009172,10.615417,11.038938,11.009222,10.216348,11.099187,11.172276,10.561021
2,TTHERM_00000010,5.066343,4.767264,5.010981,6.139047,4.619361,4.751761,5.818550,5.342529,5.483750,...,6.314438,7.423571,7.507645,7.417087,7.147801,7.747930,7.093641,7.672685,7.511290,6.890117
3,TTHERM_00000020,4.696881,4.638401,4.956299,6.942556,5.101252,4.730307,8.457690,4.526411,4.908300,...,5.250233,4.974993,5.747498,5.252167,5.210531,7.083187,5.252222,5.037613,5.495281,5.013987
4,TTHERM_00000030,4.654278,4.537105,4.928739,5.063991,4.584168,4.911880,5.935311,4.519470,4.757861,...,4.651688,4.920573,4.636333,4.883712,4.779395,4.744335,4.513140,4.838428,4.961475,4.653340


In [7]:
# full_filtered_df = shuffle_rows(full_filtered_df)
# partition_type = 'NC'

In [8]:
full_filtered_df.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_000000042,6.928782,7.264201,6.934214,6.732989,6.970612,7.150978,6.126826,6.868968,6.641119,...,6.450318,8.049750,7.788162,7.052154,6.517742,6.918501,6.048861,7.041619,6.757932,5.817246
1,TTHERM_000000045,9.633489,9.977124,10.027529,9.720665,9.605762,10.225542,10.279608,10.459966,10.693337,...,11.130466,11.207738,11.009172,10.615417,11.038938,11.009222,10.216348,11.099187,11.172276,10.561021
2,TTHERM_00000010,5.066343,4.767264,5.010981,6.139047,4.619361,4.751761,5.818550,5.342529,5.483750,...,6.314438,7.423571,7.507645,7.417087,7.147801,7.747930,7.093641,7.672685,7.511290,6.890117
3,TTHERM_00000020,4.696881,4.638401,4.956299,6.942556,5.101252,4.730307,8.457690,4.526411,4.908300,...,5.250233,4.974993,5.747498,5.252167,5.210531,7.083187,5.252222,5.037613,5.495281,5.013987
4,TTHERM_00000030,4.654278,4.537105,4.928739,5.063991,4.584168,4.911880,5.935311,4.519470,4.757861,...,4.651688,4.920573,4.636333,4.883712,4.779395,4.744335,4.513140,4.838428,4.961475,4.653340


In [9]:
full_filtered_norm_df = microarray_utils.normalize_expression_per_gene(full_filtered_df)
raw_data = full_filtered_norm_df[list(full_filtered_norm_df.columns)[1:]].values

# SCAN START

In [10]:
curr_datetime = str(datetime.now())

In [11]:
idx_labels = list(range(raw_data.shape[0]))

p_minkowski = None
# metrics = ['manhattan', 'euclidean', 'cosine'] + [f'minkowski_{str(p)}' for p in np.array([0.5, 1, 2, 3, 4, 5])]
# metrics = ['clr', 'manhattan', 'euclidean', 'cosine'] + [f'minkowski_{str(p)}' for p in np.array([0.5, 1, 2, 3, 4, 5])]
# metrics = ['clr']
metrics = ['manhattan']
n_jobs = -1
random_state = 42

In [12]:
# scan_nns = np.arange(2, 13, 1)
scan_nns = [5]
scan_nns

[5]

In [13]:
# scan_rps = np.arange(0.005, 1.1, 0.005)
# scan_rps = np.arange(0.1, 1.1, 0.1)
scan_rps = [0.030, 0.035]
scan_rps

[0.03, 0.035]

In [14]:
scan_dict = {}

In [15]:
for metric_p in metrics:
    metric_p_split = metric_p.split('_')

    metric = metric_p

    if metric_p_split[0] == 'minkowski':
        metric = metric_p_split[0]
        p_minkowski = float(metric_p_split[1])

    if metric != 'clr':
        distance_matrix = clustering_utils.compute_pairwise_distance_matrix(raw_data, metric, n_jobs, p_minkowski)
    else:
        distance_matrix = clr_dist_arr

    nn_idxs, nn_dists = clustering_utils.compute_nns(raw_data, max(scan_nns), metric, random_state, n_jobs, p_minkowski, distance_matrix)

    for idx, nn in enumerate(scan_nns):     
        print(idx+1,'of',len(scan_nns))     
        print('NNs: ', nn)

        scan_dict[nn] = {}

        scan_dict[nn]['nn_idxs'] = nn_idxs
        scan_dict[nn]['nn_dists'] = nn_dists

        nn_graph = clustering_utils.compute_umap_graph(raw_data, nn, metric, nn_idxs, nn_dists)
        scan_dict[nn]['nn_graph'] = nn_graph

        for rp in tqdm.tqdm(scan_rps):

            scan_dict[nn][rp] = {}
            
            partition = clustering_utils.compute_leiden_partition(nn_graph, rp, random_state)
            scan_dict[nn][rp]['partition'] = partition

            communities = clustering_utils.compute_communities(partition, idx_labels)
            scan_dict[nn][rp]['communities'] = communities

            sil_score = clustering_utils.compute_silhouette_score(distance_matrix, partition)
            scan_dict[nn][rp]['sil_score'] = sil_score

            modularity = clustering_utils.compute_modularity(nn_graph, communities.values())
            scan_dict[nn][rp]['modularity'] = modularity

            enrichment_df = clustering_utils.compute_enrichment(full_filtered_norm_df, partition)
            scan_dict[nn][rp]['enrichment_df'] = enrichment_df

            num_clusters = clustering_utils.compute_num_clusters(partition, communities.values())
            scan_dict[nn][rp]['num_clusters'] = num_clusters

            num_enriched_clusters = clustering_utils.compute_num_enriched_clusters(enrichment_df)
            scan_dict[nn][rp]['num_enriched_clusters'] = num_enriched_clusters

            num_enriched_cluster_genes = clustering_utils.compute_num_enriched_cluster_genes(enrichment_df, partition)
            scan_dict[nn][rp]['num_enriched_cluster_genes'] = num_enriched_cluster_genes

            cluster_sizes = clustering_utils.compute_cluster_sizes(communities)
            scan_dict[nn][rp]['cluster_sizes'] = cluster_sizes

            enriched_cluster_sizes = clustering_utils.compute_enriched_cluster_sizes(communities, enrichment_df)
            scan_dict[nn][rp]['enriched_cluster_sizes'] = enriched_cluster_sizes

            cluster_stats = {
            'partition_type': partition_type,

            'dimensionality': 'baseline',

            'metric': metric_p,
            'graph': 'umap_fuzzy_simplicial_set',
            'nns': nn,

            'clustering': 'leiden_cpm',
            'parameter': rp,

            'silhouette_score': sil_score,
            'modularity': modularity,

            'nclusters': num_clusters,
            'mean_cluster_size': clustering_utils.compute_cluster_size_mean(cluster_sizes),
            'median_cluster_size': clustering_utils.compute_cluster_size_median(cluster_sizes),
            'sd_cluster_size': clustering_utils.compute_cluster_size_sd(cluster_sizes),

            'nenriched_clusters': num_enriched_clusters,
            'mean_enriched_cluster_size': clustering_utils.compute_cluster_size_mean(enriched_cluster_sizes),
            'median_enriched_cluster_size': clustering_utils.compute_cluster_size_median(enriched_cluster_sizes),
            'sd_enriched_cluster_size': clustering_utils.compute_cluster_size_sd(enriched_cluster_sizes),
            'nenriched_cluster_genes': num_enriched_cluster_genes,

            'datetime': curr_datetime
            }

            file_utils.write_to_csv('./scan_stats_mar2024_enrich_io.csv', cluster_stats, list(cluster_stats.keys()))

1 of 1
NNs:  5


100%|██████████| 2/2 [00:41<00:00, 20.96s/it]


In [16]:
pd.read_csv('./scan_stats_mar2024_enrich_io.csv')

,partition_type,dimensionality,metric,graph,nns,clustering,parameter,silhouette_score,modularity,nclusters,mean_cluster_size,median_cluster_size,sd_cluster_size,nenriched_clusters,mean_enriched_cluster_size,median_enriched_cluster_size,sd_enriched_cluster_size,nenriched_cluster_genes,datetime
0,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.030,0.009592,0.516614,1231,16.511779,15.0,7.364789,354,17.997175,17.0,7.744690,6371,2024-03-26 12:43:09.554962
1,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.035,0.012080,0.506566,1363,14.912693,14.0,6.485754,350,15.977143,15.0,6.974611,5592,2024-03-26 12:43:09.554962
2,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.030,0.009592,0.516614,1231,16.511779,15.0,7.364789,354,17.997175,17.0,7.744690,6371,2024-03-26 12:44:57.219347
3,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.035,0.012080,0.506566,1363,14.912693,14.0,6.485754,350,15.977143,15.0,6.974611,5592,2024-03-26 12:44:57.219347
4,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.030,0.009592,0.516614,1231,16.511779,15.0,7.364789,354,17.997175,17.0,7.744690,6371,2024-03-26 13:55:05.745425
5,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.035,0.012080,0.506566,1363,14.912693,14.0,6.485754,350,15.977143,15.0,6.974611,5592,2024-03-26 13:55:05.745425


In [17]:
pd.read_csv('./scan_stats_mar2024_enrich_files.csv')

,partition_type,dimensionality,metric,graph,nns,clustering,parameter,silhouette_score,modularity,nclusters,mean_cluster_size,median_cluster_size,sd_cluster_size,nenriched_clusters,mean_enriched_cluster_size,median_enriched_cluster_size,sd_enriched_cluster_size,nenriched_cluster_genes,datetime
0,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.030,0.009592,0.516614,1231,16.511779,15.0,7.364789,354,17.997175,17.0,7.744690,6371,2024-03-26 10:42:00.850728
1,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,5,leiden_cpm,0.035,0.012080,0.506566,1363,14.912693,14.0,6.485754,350,15.977143,15.0,6.974611,5592,2024-03-26 10:42:00.850728
